# Package install

Once ssh into the cluster, install these packages by running:

- pip install "dask[complete]"==2022.11.1
- pip install seq-io
- pip install hyperspy
- pip install pyxem

# Load seq file, apply dark and gain correction

## initialize a dask job scheduler

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=40,
                      threads_per_worker=1)
client = Client(cluster)
client

The client will be assigned to a port where you can monitor the job status. You can forward that port from compute node to head node by run forward.sh as shown below, and that ssh to the port on your computer. 

Once you've entered the compute node, run: forward.sh

In [ ]:
# forward.sh

#!/bin/sh
XDG_RUNTIME_DIR=/srv/home/[name]                    # setting the runtime directory.

#Pick a random or predefined port
PORT=$(shuf -i 6000-9999 -n 1)  # Random
#PORT=8775                             #Constant port
echo serving from port $PORT   # print out 
ssh -fN -R $PORT:localhost:$PORT [name]@mscdata.msae.wisc.edu 
echo Connect to the port by running ssh -C -L $PORT:localhost:$PORT [name]@mscdata.msae.wisc.edu on your computer


## load seq file

In [7]:
import SeqIO
import numpy as np
import math
import matplotlib.pyplot as plt
import glob,os
import math
import sys
import hyperspy.api as hs
from hyperspy import io_plugins

In [8]:
folder = '/srv/home/jwei74/Electrondetection/MTFDQE/MTF_rolling_counting_200kV/rawdata/' # the folder that contains all the files of the movie
signal = SeqIO.load_folder(folder, lazy=True, chunk_shape=(200,)) # set the chunk shape based on your dataset dimension

signal = signal.inav[103430:115585].isig[256:-256,256:-256] 
# you can load part of the data set by slicing the real space coordinates use .inav, or slicing the diffraction space using .isig
signal.data # show a panel view of your data info

dask.array<getitem, shape=(12155, 512, 512), dtype=float32, chunksize=(200, 512, 512), chunktype=numpy.ndarray>

In [ ]:
# By running signal.compute(), it will start to load the chuncks into memory and do dark/gain correction.
# You can also directly run signal.save to start compute and save the result once the compute() is finished.

signal.save('/srv/home/jwei74/Electrondetection/MTFDQE/MTF_rolling_counting_200kV/chunk_to_be_count_no_gaincr_3.zspy')

# For more analysis using pyxem, please refer to the documentation and example notebooks
https://github.com/CSSFrancis/Analysis/blob/master/pyXEM-30MinTutorial.ipynb


https://github.com/pyxem/pyxem-demos.git


https://pyxem.readthedocs.io/en/latest/
